In [ ]:
################################################################################################################################
# git clone git@github.com:Diego999/pyGAT.git
################################################################################################################################
# 环境搭建：
# conda create -n GNN_GAT
# conda activate GNN_GAT

# conda install python=3.8

# https://pytorch.org/get-started/previous-versions/
# conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia
################################################################################################################################
# conda install ipykernel
# conda install platformdirs
# pip3 install ipywidgets
# pip3 install --upgrade jupyter_core jupyter_client

# python -m ipykernel install --user --name GNN_GAT
################################################################################################################################


################################################################################################################################
# pip install scipy

# sudo apt install graphviz
# pip install graphviz
################################################################################################################################